# Data for project
Test code to merge TRACE and Compustat

To do:
- replicate HHW bond data
- construct relevant COMPUSTAT data
- merge the two

In [1]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [74]:
import wrds
conn = wrds.Connection(wrds_username="santoromano")
# conn.create_pgpass_file()

Loading library list...
Done


### TRACE (Enhanced)

In [8]:
# Imports TRACE data
trace = conn.raw_sql("""
                    select * from trace.trace_enhanced
                    limit 100000
                    """, 
                    date_cols=['trd_exctn_dt'])

In [26]:
# Dataframe info including memory usage
# trace.info() 
# 1 million rows = 32mb, approx 1687 days

In [82]:
# Computes VWAP for a given data
def vwap(df):
    p = df['rptd_pr']
    q = df['entrd_vol_qt']
    return df.assign(vwap = (p*q).cumsum() / q.cumsum())

trace_vwap = trace.sort_values('trd_exctn_dt').groupby('trd_exctn_dt').apply(vwap).drop_duplicates(subset=["trd_exctn_dt"], keep="last")

In [156]:
# Stores raw data
# trace.to_pickle("raw_trace_data.pkl")

In [83]:
# Reads raw data
# trace = pd.read_pickle("raw_trace_data.pkl")
# trace

In [86]:
%%timeit
trace.sort_values('trd_exctn_dt').groupby('trd_exctn_dt').apply(vwap).drop_duplicates(subset=["trd_exctn_dt"], keep="last")

3.32 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


3.32 seconds for about 1 million rows (32mb of data), should equate to about 2 hours for just this one line (pre-optimization)